# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Dynamic-Network-Modules" data-toc-modified-id="Dynamic-Network-Modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dynamic Network Modules</a></div><div class="lev2 toc-item"><a href="#Summary" data-toc-modified-id="Summary-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Summary</a></div><div class="lev2 toc-item"><a href="#Methodology" data-toc-modified-id="Methodology-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Methodology</a></div><div class="lev1 toc-item"><a href="#Detect-Dynamic-Communities" data-toc-modified-id="Detect-Dynamic-Communities-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Detect Dynamic Communities</a></div><div class="lev2 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Construct-Modularity-Matrices" data-toc-modified-id="Construct-Modularity-Matrices-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Construct Modularity Matrices</a></div><div class="lev2 toc-item"><a href="#Run-Genlouvain-Algorithm" data-toc-modified-id="Run-Genlouvain-Algorithm-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Run Genlouvain Algorithm</a></div><div class="lev1 toc-item"><a href="#Statistics-on-Dynamic-Modules" data-toc-modified-id="Statistics-on-Dynamic-Modules-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Statistics on Dynamic Modules</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Consensus-Dynamic-Modules" data-toc-modified-id="Consensus-Dynamic-Modules-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Consensus Dynamic Modules</a></div><div class="lev2 toc-item"><a href="#Module-Allegiance-Matrix" data-toc-modified-id="Module-Allegiance-Matrix-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Module Allegiance Matrix</a></div><div class="lev2 toc-item"><a href="#Find-clusters-in-module-allegiance-matrix" data-toc-modified-id="Find-clusters-in-module-allegiance-matrix-34"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Find clusters in module allegiance matrix</a></div><div class="lev2 toc-item"><a href="#Project-module-assignments-back-on-atlas" data-toc-modified-id="Project-module-assignments-back-on-atlas-35"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Project module assignments back on atlas</a></div>

# Dynamic Network Modules

## Summary

*__Introduction__*

  * Functional organization of the brain is modular. That is, brain regions are organized into functional domains, in which they are strongly connected to each other and weakly connected to brain regions in different functional domains. Modular organization can aid a network in distributing metabolic resources more efficiently amongst nodes.
  
  * The modular organization of a network can change dynamically, perhaps to adapt to changes in network topology. Dynamic reorganization of functinoal brain modules has been associated with ageing, learning, and neurological and psychiatrical diseases.
  
  * Functional brain modules during resting state...
  
*__References__*
  - Bassett, D. S., Wymbs, N. F., Porter, M. A., Mucha, P. J., Carlson, J. M., & Grafton, S. T. (2010). Dynamic reconfiguration of human brain networks during learning. Learning, 108(18), 19. http://doi.org/10.1073/pnas.1018985108
  - Bassett, D. S., Wymbs, N. F., Rombach, M. P., Porter, M. A., Mucha, P. J., & Grafton, S. T. (2013). Task-based core-periphery organization of human brain dynamics. PLoS Computational Biology, 9(9), e1003171. http://doi.org/10.1371/journal.pcbi.1003171.
  - Bassett, D. S., Yang, M., Wymbs, N. F., & Grafton, S. T. (2015). Learning-Induced Autonomy of Sensorimotor Systems. Nature Neuroscience, 18(5), 744–751. http://doi.org/10.1038/nn.3993
  - Sporns, Betzel papers

## Methodology

*__Dynamic Community Detection__*

  * Static and dynamic community detection algorithms assign network nodes into modules by optimizing a modularity quality function, Q, such that:
  
  $Q = \frac{1}{2\mu} \sum_{ijlr}[(A_{ijl} - \gamma_l P_{ijl})\delta_{lr} + \delta_{ij} \omega_{jlr}]
  \delta(g_{il}, g_{jr})$
  
  where, $A$ is an adjacency tensor of size $N \times N \times T$ ($N$ nodes; $T$ time windows), $i$ and $j$ are the $i^\text{th}$ and $j^\text{th}$ nodes, $l$ and $r$ are the $l^\text{th}$ and $r^\text{th}$ time windows, $P$ is a network null model of $A$, $\gamma_l$ is the structural resolution parameter for time window $l$, $\omega_{jlr}$ is the temporal resolution parameter linking node $j$ from time windows $l$ and $r$, $\mu$ is the total network connection strength.
  
*__References__*
  - Bassett, D. S., Porter, M. A., Wymbs, N. F., Grafton, S. T., Carlson, J. M., & Mucha, P. J. (2013). Robust detection of dynamic community structure in networks. Chaos (Woodbury, N.Y.), 23(1), 013142. http://doi.org/10.1063/1.4790830

# Detect Dynamic Communities

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_InpData = path_PeriphData + '/e01-Dyne_FuncNetw'
path_ExpData = path_PeriphData + '/e02a-DynFuncModule-Subject'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

if not os.path.exists('./e02a-Figures'):
    os.makedirs('./e02a-Figures')        

## Generate List of Data

In [ ]:
subj_date = glob.glob('{}/Adjacency.*.npz'.format(path_InpData))

subj_ids = {}
for s_d in subj_date:
    subj, date = s_d.split('/')[-1].split('.')[1:3]
    try:
        subj_ids[subj]
    except KeyError:
        subj_ids[subj] = {}
    
    subj_ids[subj][date] = {'Adj_Path': s_d}

## Construct Modularity Matrices
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/*.ml_mod_matr.npz".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
GAMMA = 1.0
OMEGA = 1.0
RHO = OMEGA/2.0
gen_modmat = Echobase.Network.Partitioning.Module.modularity.ml_modularity_matr

for subj in subj_ids.keys():
    
    cfg_matr = []
    for d_i, date in enumerate(subj_ids[subj].keys()):
        
        # Read the input data
        df = np.load(subj_ids[subj][date]['Adj_Path'])
        
        # Concatenate the configuration matrix over multiple runs
        for cfg_ii, cfg_vec in enumerate(Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr(df['adj_matr'])):
            cfg_matr.append(cfg_vec) 
        
        df.close()
    cfg_matr = np.array(cfg_matr)
    
    # Compute the modularity matrix
    B, twomu = gen_modmat(cfg_matr, GAMMA, OMEGA, 'None')
    
    # Cache the modularity matrices
    df_modmat = np.savez("{}/{}.ml_mod_matr.npz".format(path_ExpData, subj),
                         ml_mod_matr=B, mu=twomu)

## Run Genlouvain Algorithm
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/*.module_assignment.*".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
from multiprocessing import Pool

N_SEED = 100
parallel_run = True

# Generate processing list of subjects to run
proc_list = []
for subj in subj_ids.keys():
    subj_path = "{}/{}.ml_mod_matr.npz".format(path_ExpData, subj)
    
    for nn in xrange(N_SEED):
        proc_list.append((subj, subj_path, nn+1))

# Setup helper function to map pipeline run
def _genlouvain_helper(proc_item):
    subj, inp_path, seed = proc_item
    
    # Load the file
    print(" -- Processing: {}, with Seed: {}".format(subj, seed))
    data = np.load(inp_path, mmap_mode='r')
    
    # Run gen louvain algorithm
    cvec, q = Echobase.Network.Partitioning.Module.community.genlouvain(
        data['ml_mod_matr'], limit=1000, verbose=False)
    comm_matr = cvec.reshape(-1, 112)
    
    # Cache the community detection result
    np.savez("{}/{}.module_assignment.{}.npz".format(path_ExpData, subj,
                                                     seed),
             module_assign=comm_matr, Q=q/data['mu']) 
    
if parallel_run:
    mp = Pool(6)
    mp.map(_genlouvain_helper, proc_list)
else:
    map(_genlouvain_helper, proc_list)

# Statistics on Dynamic Modules

## Generate List of Data

In [ ]:
subj_ids = np.unique([full_subj_path.split('/')[-1].split('.')[0]
                      for full_subj_path in glob.iglob('{}/*.module_assignment.*.npz'.format(path_ExpData))])

subj_seeds = {}
for subj in subj_ids:
    subj_seeds[subj] = [full_subj_path
                        for full_subj_path in glob.iglob('{}/{}.module_assignment.*.npz'.format(path_ExpData, subj))]

## Consensus Dynamic Modules

In [ ]:
for ii, (subj, seeds) in enumerate(subj_seeds.items()):
    print(" -- Processing: {}".format(subj))
    
    # Dynamic Module Allegiance
    for ix, path in enumerate(seeds):
        data = np.load(path, mmap_mode='r')
        module_assign = data['module_assign'][...]

        n_win = module_assign.shape[0]
        n_node = module_assign.shape[1]
        
        if ix==0:
            subj_dynmod_alleg = np.zeros((n_win*n_node, n_win*n_node))

        dynmod_assign = module_assign.reshape(-1)    
        module_id = np.unique(dynmod_assign)            
        for m_id in module_id:
            m_nodes = np.flatnonzero(dynmod_assign == m_id)
            n1, n2 = np.meshgrid(m_nodes, m_nodes)

            subj_dynmod_alleg[n1, n2] += 1 
    subj_dynmod_alleg /= subj_dynmod_alleg[0, 0]
    subj_dynmod_alleg[np.diag_indices_from(subj_dynmod_alleg)] = 0
    
    # Compute Static Modularity
    k = np.sum(subj_dynmod_alleg, axis=0)
    twom = np.sum(k)
    B = subj_dynmod_alleg - np.dot(k.reshape(-1, 1), k.reshape(1, -1)) / twom
    
    cvec, q = Codebase.Networks.ModuleDetection.community.genlouvain(B,
                                                                     limit=1000,
                                                                     verbose=True)
    
    # Cache the community detection result
    np.savez("{}/{}.consensus_module.npz".format(path_ExpData, subj),
             module_assign=cvec.reshape(n_win, n_node)) 

## Module Allegiance Matrix
Aggregated across all subjects, sessions, and 100 optimizations

In [ ]:
%matplotlib inline

for ii, (subj, seeds) in enumerate(subj_seeds.items()):
    print(" -- Processing: {}".format(subj))
    
    for ix, path in enumerate(seeds):
        data = np.load(path, mmap_mode='r')
        module_assign = data['module_assign']
        
        n_win = module_assign.shape[0]
        n_node = module_assign.shape[1] 
        
        if (ii == 0) and (ix == 0):
            module_alleg = np.zeros((n_node, n_node))
            
        for iw in xrange(n_win):
            module_id = np.unique(module_assign[iw, :])
            
            for m_id in module_id:
                m_nodes = np.flatnonzero(module_assign[iw, :] == m_id)
                n1, n2 = np.meshgrid(m_nodes, m_nodes)
                
                module_alleg[n1, n2] += 1
module_alleg /= module_alleg[0, 0]
module_alleg[np.diag_indices_from(module_alleg)] = 0
np.savez("{}/consensus_module_allegiance.npz".format(path_ExpData),
         module_alleg=module_alleg) 

# Plot the Module Allegiance across the cohort
plt.figure()
ax = plt.subplot(111)
ax.matshow(module_alleg, vmin=0, vmax=1, cmap='rainbow')
ax.set_axis_off()
ax.set_title('Population Module Allegiance')

## Find clusters in module allegiance matrix

In [ ]:
# Compute Static Modularity
df = np.load('{}/consensus_module_allegiance.npz'.format(path_ExpData, subj))
module_alleg = df['module_alleg']
k = np.sum(module_alleg, axis=0)
twom = np.sum(k)

# Optimize Gamma
qq = []
gamma = np.linspace(0.5, 3.0, 1000)
for gm in gamma:
    B = module_alleg - gm*np.dot(k.reshape(-1, 1), k.reshape(1, -1)) / twom
    mod_alleg_id, q = Echobase.Network.Partitioning.Module.community.genlouvain(B, limit=10000, verbose=False)
    
    qq.append(q)
qq = np.array(qq)
opt_ix = Echobase.Statistics.Optimization.curve.find_elbow(qq)
opt_gamma = gamma[opt_ix]

# Perform consensus
B = module_alleg - opt_gamma*np.dot(k.reshape(-1, 1), k.reshape(1, -1)) / twom
n_seed = 1000
mod_consensus = np.zeros((n_node, n_node))
for n_i in xrange(n_seed):
    mod_alleg_id, q = Echobase.Network.Partitioning.Module.community.genlouvain(B, limit=10000, verbose=False)

    for m_id in np.unique(mod_alleg_id):
        m_nodes = np.flatnonzero(mod_alleg_id == m_id)
        n1, n2 = np.meshgrid(m_nodes, m_nodes)

        mod_consensus[n1, n2] += 1
mod_consensus /= mod_consensus[0, 0]
mod_consensus[np.diag_indices_from(mod_consensus)] = 0

k = np.sum(mod_consensus, axis=0)
twom = np.sum(k)
B = mod_consensus - opt_gamma*np.dot(k.reshape(-1, 1), k.reshape(1, -1)) / twom
true_mod_alleg, q = Echobase.Network.Partitioning.Module.community.genlouvain(B, limit=10000, verbose=False)

# Re-ordered Module Allegiance
plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(mod_consensus[np.meshgrid(np.argsort(true_mod_alleg), np.argsort(true_mod_alleg))],
                 cmap='rainbow', vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)

# Axis Settings
ax.set_axis_off()
ax.set_xlabel('Brain Regions');
ax.set_ylabel('Brain Regions');
plt.show()

# Print out the module membership assignment
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))
for module_id in np.unique(true_mod_alleg):
    mod_ix = np.flatnonzero(true_mod_alleg == module_id)
    
    print('Module_ID: {}'.format(module_id+1))     
    print(np.unique(roi_csv['Nomenclature'][mod_ix]))

## Project module assignments back on atlas

In [ ]:
import nibabel as nib

df_atlas = nib.load('{}/Atlas/HOA112_MNI-thr0-1mm.nii.gz'.format(path_CoreData))
atlas_matr = df_atlas.get_data()
atlas_module_matr = np.zeros_like(atlas_matr)

# Map ROI labels to Module labels
for n_i in xrange(n_node):
    atlas_module_matr[np.nonzero(atlas_matr == n_i+1)] = true_mod_alleg[n_i]+1  
    
# Create new Nifti1 Image    
df_atlas_module = nib.Nifti1Image(atlas_module_matr, df_atlas.affine, df_atlas.header)
nib.save(df_atlas_module, '{}/module_allegiance.nii.gz'.format(path_ExpData))

In [ ]:
from nilearn import plotting, image

smooth_anat_img = image.smooth_img('{}/module_allegiance.nii.gz'.format(path_ExpData), 0)
display = plotting.plot_stat_map(smooth_anat_img, smooth_anat_img,
                                 cmap='Accent', black_bg=False,
                                 colorbar=True, draw_cross=False)